In [16]:
import pandas as pd
import numpy as np
import json

def load_data(file_path):
    df = pd.read_csv(file_path)
    return df

def merge_data(df1, df2, df3):
    merged_df = pd.merge(df1, df2, df3, how='outer')
    return merged_df

def convert_to_json(df):
    json_data = df.to_json(orient='records', lines=True)
    return json_data

def save_json(json_data, output_file):
    with open(output_file, 'w') as f:
        f.write(json_data)

def remove_raw_predictions(df):
    # Remove columns if "raw" is in the column names
    cols_to_drop = [col for col in df.columns if 'raw' in col]
    if cols_to_drop:
        df.drop(columns=cols_to_drop, inplace=True)
    return df

def main():
    # Load the data
    filepaths = ['test_results_with_predictions.csv', '../eval/gemini_predictions.csv', "../self_annotation/test_final.csv"]
    dfs = [load_data(fp) for fp in filepaths]

    display(dfs[2].head())

    # rename the int_score column in the second dataframe
    #dfs[1].rename(columns={'int_score': 'gemini-2.5-falsh_score'}, inplace=True)

    # Merge the dataframes
    #merged_df = merge_data(dfs[0], dfs[1])
    
    # Remove the real_label column if it exists
    #merged_df.drop(columns=['real_label'], inplace=True)

    # Remove raw predictions if they exist
    #merged_df = remove_raw_predictions(merged_df)
    #display(merged_df.head())




    # Convert to JSON
    #json_data = convert_to_json(merged_df)

    # Save to output file
    #save_json(json_data, 'output.json')

if __name__ == "__main__":
    main()

,id,text,educational_value_labels,annotator_ids,problematic_content_label_present,problematic_content_label_agreement,language_names,language_code,all_labels,all_scores,int_score
0,d4ffcabc-e77a-48bb-8bf4-735683199642,Jobannonce: Chemist\nOBS: annoncen er udløbet!...,"['None', 'None', 'None']",['a0585a5c-b72f-4c3a-a2a3-17e8e0b4ea4f'\n '85a...,False,1.0,dan_Latn,dan_Latn,"['None', 'None', 'None', 'None', 'None']","[0, 0, 0, 0, 0]",0
1,fbdbb330-dd3a-4a66-925b-01a1bbfe6ac5,Det europæiske år for færdigheder: Kommissione...,"['Basic', 'Basic', 'Minimal']",['a0585a5c-b72f-4c3a-a2a3-17e8e0b4ea4f'\n '85a...,False,1.0,dan_Latn,dan_Latn,"['Basic', 'Basic', 'Minimal', 'Minimal', 'Good']","[2, 2, 1, 1, 3]",2
2,c6e11cad-e753-4da8-a9ab-72f0e06ea398,"Det ligner ikke, at Wan-Bissaka har en fremtid...","['None', 'None', 'Basic']",['a0585a5c-b72f-4c3a-a2a3-17e8e0b4ea4f'\n '85a...,False,1.0,dan_Latn,dan_Latn,"['None', 'None', 'Basic', 'Minimal', 'None']","[0, 0, 2, 1, 0]",1
3,09a4bdba-5b5f-49b8-9530-a424ab6868a5,"Sauro Maule 2017 ""Pri"" Garganega\nAromatisk or...","['Basic', 'None', 'None']",['a0585a5c-b72f-4c3a-a2a3-17e8e0b4ea4f'\n '740...,False,1.0,dan_Latn,dan_Latn,"['Basic', 'None', 'None', 'Minimal', 'None']","[2, 0, 0, 1, 0]",1
4,d4e44cf0-1147-46ce-9280-e82d2589a5e3,fyens.dk bruger cookies til at give dig en god...,"['None', 'None', 'None']",['a0585a5c-b72f-4c3a-a2a3-17e8e0b4ea4f'\n '998...,False,1.0,dan_Latn,dan_Latn,"['None', 'None', 'None', 'None', 'None']","[0, 0, 0, 0, 0]",0
